In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/london-bike-sharing-dataset/london_merged.csv', parse_dates=['timestamp']) #parse_dates: 시계열 인식 옵션
df.head()


외부변수 없이 자전거 이용객수 데이터 자체만의 패턴을 이용해서 딥러닝으로 학습을 시켜서 예측하는 시계열 딥러닝을 해보자


In [3]:
df['timestamp']

In [4]:
df.shape #구조

In [5]:
# 앞에서부터 17000개를 학습용으로 사용하고 나중 데이터인 414개의 시간데이터를 테스트용으로 사용하자

train = df.iloc[:17000, 1:2] #첫번째에서 두번째 사이에 있는 열 = cnt(자전거이용객수)
test = df.iloc[17000:17414, 1:2]

In [6]:
# 잘 분리 되었는지 확인

print(train.shape)
print(test.shape)

In [7]:
# 분리되어있는 데이터를 그래프를 이용하여 확인해보기

df['cnt'][:17000].plot(figsize=(15,4), legend=True)
df['cnt'][17000:].plot(figsize=(15,4), legend=True)
plt.legend(['train', 'test'])
plt.title('bike share demand')
plt.show()

- 시계열 딥러닝은 일반 딥러닝과 다름 
- 데이터 구조부터 다름
- 입력되는 독립변수가 없음
- 데이터 자체만의 패턴으로 학습해야함

In [8]:
# 데이터 전처리

# 시계열 차수를 추정하기

from statsmodels.tsa.stattools import pacf #pacf 편자기상관함수

pacf = pacf(df['cnt'], nlags=20, method='ols')
print(pacf)

In [12]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(pacf, lags=9, method='ols', title='pa').show()

- 파란색 영역을 보면 0에서 1로 갈수록 급감함
- 이를 통해 1시간 전 데이터를 독립변수 개념으로 해서 그 다음시간을 예측하는 모형을 만들어서 진행하자

In [14]:
# 데이터 스케일 작업

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1)) #0~1사이의 값을 가지도록
train_scaled = sc.fit_transform(train) #train 데이터를 트랜스폼

In [15]:
train_scaled

In [16]:
# 시계열 딥러딩: 자기 자신의 과거를 독립변수로 활용
# 차수 추정을 1로 했음 -> 1시간 전 데이터를 독립변수로 만드는 작업 필요

# train 데이터를 1시간 단위로 shift하면 그것이 독립변수가 됨

X_train = []
y_train = []

for i in range(1, 17000):
    X_train.append(train_scaled[i-1:i, 0]) #1시간 전을 X_train에 넣어줌
    y_train.append(train_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

- 독립변수, 종속변수 생성된 상황

In [17]:
# 시계열 딥러닝은 3차원의 배열을 필요로함

#현재
X_train.shape #2차원의 배열

In [19]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) #X_train.shape[0]: 16999 #X_train.shape[1]: 1
X_train.shape #3차원의 배열

In [22]:
# RNN

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN

In [23]:
#모형 설계

rnn = Sequential()
rnn.add(SimpleRNN(activation='relu', units=6, input_shape=(1,1)))
rnn.add(Dense(activation='linear', units=1))

In [24]:
print(rnn.summary())

- 6개의 유닛, 출력층은 1개, 파라미터는 48개

In [25]:
rnn.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [26]:
rnn.fit(X_train, y_train, batch_size=1, epochs=2) #X_train 자전거 이용객 쉬프트된 한시간 전 데이터

- 배치사이즈는 1, 반복횟수는 2(전체데이터 2번 학습)
- 배치사이즈를 1로 설정하였기 때문에 1단위로 학습중

In [27]:
inputs = sc.transform(test)
inputs.shape

In [28]:
X_test = []
for i in range(1, 415):
    X_test.append(inputs[i-1:i, 0])
    
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test.shape #3차원 배열로 만들었음

In [29]:
# 예측

rnn = rnn.predict(X_test)
rnn = sc.inverse_transform(rnn) #MinMaxScale 한거 원래대로 돌려놓음

In [30]:
# 그래프로 확인해보기

# 데이터프레임 형태여야 그래프로 표현 가능

test1 = pd.DataFrame(test)
rnn1 = pd.DataFrame(rnn)

In [31]:
test1.plot(figsize=(15,4), legend=True)
plt.legend(['cnt'])
plt.title('bike share demand')
plt.show() #자전거 이용객 수 실제값

In [32]:
rnn1.plot(figsize=(15,4), legend=True)
plt.legend(['cnt'])
plt.title('bike share demand')
plt.show() #자전거 이용객 수 예측값

- 예측과 실제가 유사함

In [34]:
# 위의 두 그래프를 한 그래프에 표현하기

test = np.array(test)

plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='cnt', color='black')
plt.plot(rnn, marker='.', label='RNN', color='red')
plt.legend()

- 상당히 fitting이 잘 된 모습
- RNN 모형이 피크타임 때 과소 예측하는 모습을 보임
- 시각화 참고https://datascienceschool.net/01%20python/05.01%20%EC%8B%9C%EA%B0%81%ED%99%94%20%ED%8C%A8%ED%82%A4%EC%A7%80%20%EB%A7%B7%ED%94%8C%EB%A1%AF%EB%A6%AC%EB%B8%8C%20%EC%86%8C%EA%B0%9C.html

In [55]:
# LSTM
# 기존 RNN 단점 보완 # 장기메모리

from keras.layers.recurrent import LSTM

# 기본적인 모형 구조

lstm = Sequential()
lstm.add(LSTM(units=6, activation='relu', input_shape=(1,1)))
lstm.add(Dense(units=1, activation='linear'))

In [56]:
lstm.summary()

- RNN과 비교해보면, RNN의 경우 파라미터의 수가 48개인 반면에 LSTM은 192개 임
- 이는 LSTM이 좀 더 정교하게 학습을 할 수 있다는 것. 그러나 과적합이 발생할 수 있다는 단점이 있음

In [57]:
lstm.compile(loss='mse', optimizer='adam', metrics=['mse'])
lstm.fit(X_train, y_train, batch_size=1, epochs=2)

In [46]:
X_test.shape

In [58]:
# 예측

lstm = lstm.predict(X_test)
lstm = sc.inverse_transform(lstm) #MinMaxScale 한거 원래대로 돌려놓음

In [59]:
plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='cnt', color='black')
plt.plot(lstm, marker='.', label='LSTM', color='green')
plt.legend()

- 거의 다 맞춘 모습을 보임

In [60]:
# GRU

# LSTM의 과적합문제를 보완해주는 모형

from keras.layers.recurrent import GRU

gru = Sequential()
gru.add(GRU(units=6, activation='relu', input_shape=(1,1)))
gru.add(Dense(units=1, activation='linear'))

In [61]:
print(gru.summary())

- 파라미터의 수가 RNN보다는 많고, LSTM보다는 적음
- 과대적합 발생을 줄임

In [62]:
gru.compile(loss='mse', optimizer='adam', metrics=['mse'])
gru.fit(X_train, y_train, batch_size=1, epochs=2)

In [63]:
# 예측

gru = gru.predict(X_test)
gru = sc.inverse_transform(gru) #MinMaxScale 한거 원래대로 돌려놓음

In [64]:
plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='cnt', color='black')
plt.plot(gru, marker='.', label='GRU', color='blue')
plt.legend()

In [65]:
# 세개의 모형 비교하기

# 모형별 차이 알아보기

plt.figure(figsize=(15,5))
plt.plot(test, marker='.', label='cnt', color='black')
plt.plot(rnn, marker='.', label='RNN', color='red')
plt.plot(lstm, marker='.', label='LSTM', color='green')
plt.plot(gru, marker='.', label='GRU', color='blue')
plt.legend()

- LSTM의 fitting이 제일 좋아 보임

In [66]:
# RMSE를 이용하여 수치적으로 비교해보기

from sklearn.metrics import mean_squared_error

def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))

In [67]:
print('RNN RMSE:', RMSE(test,rnn))
print('LSTM RMSE:', RMSE(test,lstm))
print('GRU RMSE:', RMSE(test,gru))

- epochs를 2번 밖에 안한 상태
- 파라미터 조정에 따라서 최적의 모델이 바뀔 가능성이 존재함
- 일단, 지금까지 가장 좋은 모델은 순서대로 LSTM > GRU > RNN